In [3]:
import pandas as pd
import numpy as np
import requests
import re
import os
from time import sleep
from tqdm import tqdm
from functools import reduce

import datetime

In [8]:
BASE_GOOGLE_FINANCE_URL = "https://api.polygon.io/v2/aggs/ticker/{TICKER}/range/{INTERVAL}/" +\
                          "{RANGE}/{FROM_DATE}/{TO_DATE}?limit=50000&apiKey=MNxlbyNXPpKmAki1jcScqmXxzPvdXWYT"

TRAIN_RESULT_PATH = "data/prices/google_30m_prices_train.csv"
TEST_RESULT_PATH = "data/prices/google_30m_prices_test.csv"

TOKENS = ["AAPL", "GOOG", "MSFT", "TSLA", "NVDA"]

TIME_INTERVAL = 15
TIME_UNITS = "minute"
FROM_DATE = pd.Timestamp("2019-02-18").date()
TO_DATE = pd.Timestamp("2022-03-28").date()

In [9]:
def get_url(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    """
    Build url for Google Finance API
    """
    url = re.sub("{TICKER}", token, BASE_GOOGLE_FINANCE_URL)
    url = re.sub("{INTERVAL}", str(interval_len), url)
    url = re.sub("{RANGE}", interval, url)
    url = re.sub("{FROM_DATE}", str(from_date), url)
    url = re.sub("{TO_DATE}", str(to_date), url)
    return url

In [10]:
def get_data(token: str, interval_len: int, interval: str, from_date: datetime.date, to_date: datetime.date):
    TIMEOUT = 20
    current_date = from_date
    previous_date = current_date
    dfs = []
    while current_date <= to_date:
        request_url = get_url(token, interval_len, interval, current_date, to_date)
        data = requests.get(url=request_url)
        if data.json()["status"] == "ERROR":
            print("Waiting for", TIMEOUT, "seconds")
            for _ in tqdm(range(TIMEOUT)):
                sleep(1)
        else:
            try:
                current_df = pd.DataFrame(data.json()["results"])
            except:
                pass
            current_date = (pd.Timestamp(current_df["t"].max(), unit="ms") + pd.DateOffset(days=1)).date()
            print("Current date:", current_date)
            print("Get DataFrame of shape", current_df.shape)
            dfs.append(current_df)
    result_df = pd.concat(dfs)
    result_df.drop_duplicates(inplace=True)
    return result_df

In [ ]:
dfs = []
for token in TOKENS:
    data = get_data(
        token=token,
        interval_len=TIME_INTERVAL,
        interval=TIME_UNITS,
        from_date=FROM_DATE, 
        to_date=TO_DATE
    )

    data["t"] = data["t"].apply(lambda row: pd.Timestamp(row, unit="ms"))
    data["ticker"] = token
    dfs.append(data)

Current date: 2020-07-08
Get DataFrame of shape (4378, 8)
Current date: 2020-10-02
Get DataFrame of shape (3864, 8)
Current date: 2020-12-24
Get DataFrame of shape (3644, 8)
Current date: 2021-03-19
Get DataFrame of shape (3620, 8)
Current date: 2021-06-17
Get DataFrame of shape (3920, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2021-09-17
Get DataFrame of shape (4052, 8)
Current date: 2021-12-15
Get DataFrame of shape (3925, 8)
Current date: 2022-03-15
Get DataFrame of shape (3843, 8)
Current date: 2022-03-26
Get DataFrame of shape (575, 8)
Current date: 2022-03-26
Get DataFrame of shape (575, 8)
Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


...

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Waiting for 20 seconds


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:20<00:00,  1.01s/it]


Current date: 2022-03-26
Get DataFrame of shape (575, 8)
Current date: 2022-03-26
Get DataFrame of shape (575, 8)
Current date: 2022-03-26
Get DataFrame of shape (575, 8)
Current date: 2022-03-26
Get DataFrame of shape (575, 8)
Current date: 2022-03-26
Get DataFrame of shape (575, 8)
Waiting for 20 seconds


 10%|████████▎                                                                          | 2/20 [00:02<00:18,  1.01s/it]

In [41]:
TEST_RATIO = 0.2
test_size = int(data.shape[0] * TEST_RATIO)
data_train = data.iloc[:-test_size]
data_test = data.iloc[-test_size:]

data_train.to_csv(TRAIN_RESULT_PATH)
data_test.to_csv(TEST_RESULT_PATH)